In [ ]:
#Iterate Through paging
import requests
from bs4 import BeautifulSoup

# Base URL for the pages
base_url = "https://sinta.kemdikbud.go.id/affiliations/authors/400?page="

# Function to scrape a single page
def scrape_page(page_number):
    url = base_url + str(page_number)
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        profile_ids = soup.find_all('div', class_='profile-id')
        profile_names = soup.find_all('div', class_='profile-name')
        profile_departments = soup.find_all('div', class_='profile-dept')
    
        
        for profile_id, profile_name,profile_dept in zip(profile_ids, profile_names, profile_departments):
            # Extract the ID number
            id_text = profile_id.get_text(strip=True)
            id_number = id_text.split(":")[1].strip()
            
            # Extract the author name
            author_name = profile_name.find('a').get_text(strip=True)

            # Extract the department
            profile_dept = profile_dept.find('a').get_text(strip=True)
            
            print(f"ID: {id_number}, Name: {author_name}, Department: {profile_dept}")
    else:
        print(f"Failed to retrieve page {page_number}. Status code: {response.status_code}")

# Iterate through pages (1 to 120)
for page in range(1, 121):
    scrape_page(page)

ID: 5975443, Name: NUR ROHIM YUNUS, Department: Ilmu Hukum (S1)
ID: 6687811, Name: MAILA DINIA HUSNI RAHIEM, Department: Pendidikan Islam Anak Usia Dini (S1)
ID: 56012, Name: MOHAMMAD NUR RIANTO AL ARIF, Department: Ekonomi Syariah (S2)
ID: 6674970, Name: MUHAMAD ALI, Department: Unknown
ID: 6198682, Name: SITI UMMI MASRUROH, Department: Teknik Informatika (S1)
ID: 56477, Name: AANG SUBIYAKTO, Department: Teknologi Informasi (S2)
ID: 6022882, Name: DEWI KHAIRANI, Department: Teknik Informatika (S1)
ID: 6691239, Name: DIDIN NURUDDIN HIDAYAT, Department: Pendidikan Bahasa Inggris (S2)
ID: 6005115, Name: MUNASPRIANTO RAMLI, Department: Tadris Kimia (S1)
ID: 5974678, Name: HUSNI TEJA SUKMANA, Department: Teknik Informatika (S1)
ID: 6108706, Name: IMAM SUBCHI, Department: Sejarah dan Peradaban Islam (S1)
ID: 6198158, Name: MUHBIB, Department: Pendidikan Bahasa Arab (S2)
ID: 6765599, Name: AGUS RIMUS LIANDI, Department: Kimia (S1)
ID: 6125090, Name: ZULKIFLI, Department: Sosiologi (S2)
ID: 6

KeyboardInterrupt: 

In [ ]:
#Extract author contributions
import requests
from bs4 import BeautifulSoup

# List of author IDs (replace with your actual list of IDs)
author_ids = [
    "6687811",  # Add all your author IDs here
    # "another_author_id",
    # ...
]

# Base URL for the author profile pages
base_url = "https://sinta.kemdikbud.go.id/authors/profile/"

# Function to scrape a single author page with a specific view
def scrape_author_page(author_id, view):
    url = f"{base_url}{author_id}/?view={view}"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        meta_index = 1 if view == "scopus" else 2
        
        # Extract the desired information from divs with class 'ar-list-item'
        items = soup.find_all('div', class_='ar-list-item')
        
        for item in items:
            # Extract the title inside the item
            title_element = item.find('div', class_='ar-title')
            title_text = title_element.get_text(strip=True)
            # Extract Published Date
            meta_elements = item.find_all('div', class_='ar-meta')
            year_text = meta_elements[meta_index].find('a').get_text(strip=True)
            
            print(f"Author ID: {author_id}, View: {view}, Title: {title_text}, Date: {year_text}")
        
    else:
        print(f"Failed to retrieve author page {author_id} with view {view}. Status code: {response.status_code}")

# Iterate through the list of author IDs and scrape each author page for different views
views = ["scopus", "researches", "books"]
for author_id in author_ids:
    for view in views:
        scrape_author_page(author_id, view)

Author ID: 6687811, View: scopus, Title: Early childhood education contingencies for sustaining learning during school closures: Lessons from preschool remote education home visits in Indonesia during the COVID-19 pandemic, Date: 2024
Author ID: 6687811, View: scopus, Title: Religion, Education, Science and Technology towards a More Inclusive and Sustainable Future: Proceedings of the 5th International Colloquium on Interdisciplinary Islamic Studies (ICIIS 2022), Lombok, Indonesia, 19-20 October 2022, Date: 2024
Author ID: 6687811, View: scopus, Title: Enhancing Student Engagement in Computational Physics through Project-Based Learning: An Exploration of VBA Physics Simulation, Date: 2023
Author ID: 6687811, View: scopus, Title: Science in Islamic kindergarten in Indonesia: Content and methods, Date: 2023
Author ID: 6687811, View: scopus, Title: Exploring how and what math is taught in Islamic kindergarten in Indonesia, Date: 2023
Author ID: 6687811, View: scopus, Title: Student Percep

KeyboardInterrupt: 

### Final Code

In [10]:
import sqlite3
import requests
from bs4 import BeautifulSoup

# Create a connection to the SQLite database
conn = sqlite3.connect('result.db')
cursor = conn.cursor()

# Create a table to store the scraped data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS contributions (
        id TEXT,
        name TEXT,
        department TEXT,
        view TEXT,
        title TEXT,
        date TEXT
    )
''')

# Function to insert data into the database
def insert_data(id_number, author_name, profile_dept, view, title_text, year_text):
    cursor.execute('''
        INSERT INTO contributions (id, name, department, view, title, date)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (id_number, author_name, profile_dept, view, title_text, year_text))
    conn.commit()

def close_connection():
    conn.close()

# Function to scrape a single author page with a specific view
def scrape_author_page(author_id, author_name, profile_dept, view):
    # Base URL for the author profile pages
    base_url = "https://sinta.kemdikbud.go.id/authors/profile/"
    url = f"{base_url}{author_id}/?view={view}"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        meta_index = 1 if view == "scopus" else 2
        
        # Extract the desired information from divs with class 'ar-list-item'
        items = soup.find_all('div', class_='ar-list-item')
        results = []
        
        for item in items:
            # Extract the title inside the item
            title_element = item.find('div', class_='ar-title')
            title_text = title_element.get_text(strip=True)
            # Extract Published Date
            meta_elements = item.find_all('div', class_='ar-meta')
            year_text = meta_elements[meta_index].find('a').get_text(strip=True)
            insert_data(author_id, author_name, profile_dept, view, title_text, year_text)        
    else:
        print(f"Failed to retrieve author page {author_id} with view {view}. Status code: {response.status_code}")

# Function to scrape author contributions and insert them into the database
def scrape_page(page_number):
    # Base URL for the pages
    base_url = "https://sinta.kemdikbud.go.id/affiliations/authors/400?page="
    url = base_url + str(page_number)
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        profile_ids = soup.find_all('div', class_='profile-id')
        profile_names = soup.find_all('div', class_='profile-name')
        profile_departments = soup.find_all('div', class_='profile-dept')
    
        
        for profile_id, profile_name,profile_dept in zip(profile_ids, profile_names, profile_departments):
            # Extract the ID number
            id_text = profile_id.get_text(strip=True)
            id_number = id_text.split(":")[1].strip()
            
            # Extract the author name
            author_name = profile_name.find('a').get_text(strip=True)

            # Extract the department
            profile_dept = profile_dept.find('a').get_text(strip=True)
            # Iterate through the list of author IDs and scrape each author page for different views
            views = ["scopus", "researches", "books"]
            for view in views:
                scrape_author_page(id_number, author_name, profile_dept, view)
                
    else:
        print(f"Failed to retrieve page {page_number}. Status code: {response.status_code}")

# Iterate through pages (1 to 120)
for page in range(1, 121):
    scrape_page(page)

In [11]:
#print current time 10.25
import datetime
print(datetime.datetime.now())

2024-11-01 11:09:07.669738


In [ ]:
# Data Collected On 10.25 - 11.09